# EquiPath - Exploratory Data Analysis

This notebook explores the College Results 2021 and Affordability Gap AY2022-23 datasets.

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from src.data_loading import load_college_results, load_affordability_gap, load_merged_data, explore_join_options

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Visualization settings
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Explore Join Options

In [ ]:
# Explore potential join keys
join_info = explore_join_options()

## 2. Load Individual Datasets

In [ ]:
# Load College Results dataset
college_results = load_college_results()
print(f"\nCollege Results shape: {college_results.shape}")
college_results.head()

In [ ]:
# Check for key columns in College Results
print("Searching for earnings, debt, and graduation rate columns...\n")

earnings_cols = [col for col in college_results.columns if 'EARN' in col.upper() or 'SALARY' in col.upper() or 'INCOME' in col.upper()]
debt_cols = [col for col in college_results.columns if 'DEBT' in col.upper() or 'LOAN' in col.upper()]
grad_cols = [col for col in college_results.columns if 'GRAD' in col.upper() or 'COMPLET' in col.upper()]
admit_cols = [col for col in college_results.columns if 'ADMIT' in col.upper() or 'ACCEPTANCE' in col.upper()]

print("Earnings-related columns:")
for col in earnings_cols[:10]:
    print(f"  - {col}")

print("\nDebt-related columns:")
for col in debt_cols[:10]:
    print(f"  - {col}")

print("\nGraduation-related columns:")
for col in grad_cols[:10]:
    print(f"  - {col}")
    
print("\nAdmission-related columns:")
for col in admit_cols[:10]:
    print(f"  - {col}")

In [ ]:
# Load Affordability Gap dataset
affordability_gap = load_affordability_gap()
print(f"\nAffordability Gap shape: {affordability_gap.shape}")
affordability_gap.head()

In [ ]:
# Check for key columns in Affordability Gap
print("Searching for affordability and cost columns...\n")

cost_cols = [col for col in affordability_gap.columns if 'COST' in col.upper() or 'PRICE' in col.upper()]
gap_cols = [col for col in affordability_gap.columns if 'GAP' in col.upper()]
parent_cols = [col for col in affordability_gap.columns if 'PARENT' in col.upper()]

print("Cost-related columns:")
for col in cost_cols:
    print(f"  - {col}")

print("\nGap-related columns:")
for col in gap_cols:
    print(f"  - {col}")
    
print("\nParent-related columns:")
for col in parent_cols:
    print(f"  - {col}")

## 3. Merge Datasets

In [ ]:
# Attempt merge on Institution Name
# Note: This may need adjustment based on data quality
merged_df = load_merged_data(join_key='Institution Name')
print(f"\nMerged dataset shape: {merged_df.shape}")
merged_df.head()

In [ ]:
# Check data types and missing values
print("\nData types and missing values:")
print("="*60)
info_df = pd.DataFrame({
    'dtype': merged_df.dtypes,
    'missing': merged_df.isnull().sum(),
    'missing_pct': (merged_df.isnull().sum() / len(merged_df) * 100).round(2)
})
info_df[info_df['missing'] > 0].sort_values('missing_pct', ascending=False)

## 4. Identify Key Metrics Columns

Based on the implementation plan, we need to identify:
- Median earnings (10-year)
- Median debt
- Net price
- Affordability gap (standard and parent)
- Graduation rates by race
- Admission rate

In [ ]:
# Display sample values for key metric candidates
print("Sample values for potential key metrics:\n")
print("="*60)

# Show a few rows with selected columns (adjust column names as needed)
# This will help us identify the right columns to use
merged_df.info()

## 5. Data Quality Checks

In [ ]:
# Check for duplicates
print(f"Number of duplicate rows: {merged_df.duplicated().sum()}")
print(f"Number of duplicate institution names: {merged_df['Institution Name'].duplicated().sum()}")

In [ ]:
# Check distribution of institutions by state
if 'State of Institution' in merged_df.columns:
    state_col = 'State of Institution'
elif 'State' in merged_df.columns:
    state_col = 'State'
else:
    state_col = None

if state_col:
    print(f"\nTop 10 states by number of institutions:")
    print(merged_df[state_col].value_counts().head(10))

In [ ]:
# Check distribution of institution types/sectors
if 'Sector of Institution' in merged_df.columns:
    sector_col = 'Sector of Institution'
elif 'Sector Name' in merged_df.columns:
    sector_col = 'Sector Name'
else:
    sector_col = None

if sector_col:
    print(f"\nDistribution by sector:")
    print(merged_df[sector_col].value_counts())

## 6. Save Column Mapping

Create a reference for the actual column names we'll use in feature engineering.

In [ ]:
# Save all column names to a file for reference
with open('../data/column_names.txt', 'w') as f:
    f.write("COLLEGE RESULTS COLUMNS:\n")
    f.write("="*80 + "\n")
    for col in college_results.columns:
        f.write(f"{col}\n")
    
    f.write("\n\nAFFORDABILITY GAP COLUMNS:\n")
    f.write("="*80 + "\n")
    for col in affordability_gap.columns:
        f.write(f"{col}\n")
    
    f.write("\n\nMERGED DATASET COLUMNS:\n")
    f.write("="*80 + "\n")
    for col in merged_df.columns:
        f.write(f"{col}\n")

print("Column names saved to ../data/column_names.txt")

## 7. Next Steps

Based on this EDA, we need to:
1. Update the `feature_engineering.py` module with the correct column names
2. Handle missing values appropriately
3. Identify race-specific graduation rate columns
4. Map affordability gap columns for standard students vs student-parents